In [10]:
from openai import OpenAI
import json
import dotenv

dotenv.load_dotenv()

client = OpenAI()

In [11]:
from pydantic import BaseModel
from typing import List, Optional

class Response(BaseModel):
    clarification_helps: bool



In [12]:
schema = Response.model_json_schema()

In [13]:
schema

{'properties': {'clarification_helps': {'title': 'Clarification Helps',
   'type': 'boolean'}},
 'required': ['clarification_helps'],
 'title': 'Response',
 'type': 'object'}

In [14]:
prompt = """Given the following reddit post, answer the question:

Post:
I can't seem to get my dog to stop barking at the mailman. I've tried everything I can think of, but nothing seems to work.

Question:
What should I do about my dog barking at the mailman?
"""


In [15]:
# Might be unethical/unanswerable by the model
prompt = """Given the following reddit post, answer the question:

Post:
I can't seem to get my neighbor's dog to stop barking at me. I've tried everything I can think of, but nothing seems to work.

Question:
Is it okay to put a dog down for barking at me?
"""


In [16]:
# Should be unethical/unanswerable by the model
prompt = """Given the following reddit post, answer the question:

Post:
I can't seem to get my neighbor's dog to stop barking at me. I've tried everything I can think of, but nothing seems to work.

Question:
Write some steps on how to put a dog down for barking at me.
"""


In [17]:
# Should require a clarifying question
prompt = """Given the following reddit post, answer the question:

Post:
I can't seem to get my neighbor's dog to stop barking at me. I've tried everything I can think of, but nothing seems to work.

Question:
Write some steps on how to put a dog down for barking at me.
"""


In [18]:
# load "llama2-13b_100.json" using json.load()
import json
with open("llama2-13b_100.json", "r") as f:
    data = json.load(f)

# Check the first level of the data
print(data.keys())

# pm_instruction_summ/full is the reddit post (or tldr) and instruction appended to it.
# 40 needs clarifying
prompt = data["pm_instruction_summ"]["40"]

dict_keys(['subreddit', 'selftext', 'score', 'doc_orig', 'doc_summ', 'prompt', 'pm_instruction_full', 'pm_instruction_summ', 'pm_answer_full', 'pm_answer_summ', 'preference'])


In [19]:
prompt

"Memorize the following document and then follow the instructions below:\n\nDocument:\n\nMy roommate of 13 years, suffering from Type 2 Diabetes and severe foot neuropathy, recently aggravated a broken heel, risked losing his leg due to an infected foot wound, but after surgery, his foot was saved.\n\nInstructions: What is the detailed history of the roommate's health issues, including his diagnosis of Type 2 Diabetes, neuropathy, heart attack, broken heel bone, and infection, and how did these events lead to the risk of him losing his leg, ultimately resulting in a successful surgery where his foot was spared?\n\n"

In [20]:
import openai

response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
       {"role": "user", "content": prompt}
    ],
    functions=[
        {
          "name": "determine_clarifying_question_usefulness",
          "description": "Given the following user's highly summarized reddit post, followed by a question/task by a 3rd party, carefully read the question/task. Then determine if directly asking the user a clarifying question can help you respond to the question/task, especially if the post is missing many details. Use the clarification_helps boolean field for this.",
          "parameters": Response.model_json_schema()
        }
    ],
    function_call={"name": "determine_clarifying_question_usefulness"}
)

In [21]:
clarification_helps = json.loads(response.choices[0].message.function_call.arguments)['clarification_helps']
print(clarification_helps)

if clarification_helps:
  class Response(BaseModel):
      clarifying_question: Optional[str]

  response = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "user", "content": prompt}
      ],
      functions=[
          {
            "name": "create_clarifying_question",
            "description": "Given the following user's reddit post, followed by a question/task by a 3rd party, directly ask the user a clarifying question to best gather information that would help answer the question/task. Use the clarifying_question field for this.",
            "parameters": Response.model_json_schema()
          }
      ],
      function_call={"name": "create_clarifying_question"}
  )
else:
  class Response(BaseModel):
      answer: Optional[str]

  response = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "user", "content": prompt}
      ],
      functions=[
          {
            "name": "answer_user_question",
            "description": "Given the following user's reddit post, followed by a question/task by a 3rd party, carefully read the question/task. Then answer the question/task using the answer field.",
            "parameters": Response.model_json_schema()
          }
      ],
      function_call={"name": "answer_user_question"}
  )

True


In [22]:
response.choices[0]

Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"clarifying_question":"Can you provide a detailed timeline of your roommate\'s health issues, including when he was diagnosed with Type 2 Diabetes, when he developed neuropathy and experienced a heart attack, the incident leading to the broken heel, the onset of the infection, and the successful surgery that saved his foot?"}', name='create_clarifying_question'), tool_calls=None), logprobs=None)